In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key= "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",
    model="llama-3.3-70b-versatile"

)
response = llm.invoke("The First person to land on Moon?")
print(response.content)

The first person to land on the Moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the Moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the Moon.


In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/data-engineer/job/R-74933")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.






















Data Engineer












































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



Select a La

In [3]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    The scraped text is from the carrer's page of a website.
    Your job is to extract the job postings and return them in JSON format containing following keys:
    'role', 'experience', 'skills', and 'description'.
    only return the valid JSON.
    ### VALID JSON(NO PREAMBLE):
    """
)
chain_extract = prompt_extract | llm
response = chain_extract.invoke(input={"page_data": page_data})
type(response.content)



str

In [4]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
parsed_response = json_parser.parse(response.content)
print(parsed_response)


{'role': 'Data Engineer', 'experience': '5 years of progressive, post-baccalaureate experience in job offered or in a engineering-related-occupation', 'skills': ['Python', 'Airflow', 'SQL', 'Spark', 'Cloud – AWS/Azure', 'Databricks', 'Snowflake', 'Data Pipeline Design'], 'description': 'Build and deliver scalable data and analytics solutions focused on Consumer and Marketplace data products; design, implement and integrate new technologies and evolve data and analytics products; contribute to all aspects of data engineering from ingestion, transformation, and consumption in addition to designing and building test-driven development, reusable frameworks, automated workflows, and libraries at scale to support analytics products'}


In [5]:
import pandas as pd 
df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [6]:
import uuid
import chromadb

cilent = chromadb.PersistentClient('Vectorstore')
collection = cilent.get_or_create_collection(name='portfolio')

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())]
        )
        



In [11]:
job = parsed_response
job['skills']

['Python',
 'Airflow',
 'SQL',
 'Spark',
 'Cloud – AWS/Azure',
 'Databricks',
 'Snowflake',
 'Data Pipeline Design']

In [12]:
links = collection.query(
    query_texts=job['skills'], n_results=2).get('metadatas', [])
    
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/kotlin-android-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/kotlin-android-portfolio'}],
 [{'links': 'https://example.com/android-tv-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}]]

In [15]:
prompt_email = PromptTemplate.from_template(

    
"""
### JOB DESCRIPTION
### INSTRUCTION

You are Vennela, a business development executive at AtliQ. AtliQ ia an AI & Software development for 
the seamless integration of business processes through automated tools.
Over our experience, we have empowered numerous enterprises with tailored solutions and process 
optimization,cost reduction, and heightened overall effiency.
Your job is to write a cold email to the Hiring Manager regarding the job mentioned above in fulfilling their needs.
Also add the most relevant ones from the following links to showcase AtliQ's portfolio.
Remember you are vennela, BDE at AtliQ.
Do not provide a preamble.
### EMAIL(NO PREAMBLE):
"""
)

chain_email = prompt_email | llm
response = chain_email.invoke({"job_description": str(job), "link_list": links})
print(response.content)

Subject: Unlock Seamless Business Process Integration with AtliQ

Dear Hiring Manager,

I came across your company and was impressed by the innovative work you're doing. As a Business Development Executive at AtliQ, I'd like to introduce you to our AI and software development capabilities that can help streamline your business processes, reduce costs, and boost overall efficiency.

At AtliQ, we've had the privilege of empowering numerous enterprises with tailored solutions, resulting in significant process optimization and cost reduction. Our expertise lies in developing automated tools that integrate seamlessly with existing systems, enabling businesses to operate more efficiently.

Some notable examples of our work include:

* **Automating Accounts Payable Process for a Leading Retailer**: We developed an AI-powered accounts payable automation tool that reduced processing time by 70% and increased accuracy by 90%. (https://atliq.com/case-study/automating-accounts-payable-process/)
* 